# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.123762e+02     1.728386e+00
 * time: 0.07359004020690918
     1     1.108607e+01     9.583937e-01
 * time: 1.3999600410461426
     2    -1.218139e+01     1.098011e+00
 * time: 1.498365879058838
     3    -3.401926e+01     8.550176e-01
 * time: 1.6666450500488281
     4    -4.768162e+01     7.008828e-01
 * time: 1.7826788425445557
     5    -5.691913e+01     2.847859e-01
 * time: 1.9039690494537354
     6    -5.965968e+01     2.487985e-01
 * time: 2.0047600269317627
     7    -6.080247e+01     1.242613e-01
 * time: 2.1194729804992676
     8    -6.133402e+01     3.725013e-02
 * time: 2.2082579135894775
     9    -6.160275e+01     3.963403e-02
 * time: 2.2980589866638184
    10    -6.179753e+01     4.039571e-02
 * time: 2.388746976852417
    11    -6.195890e+01     2.539525e-02
 * time: 2.4799108505249023
    12    -6.206463e+01     1.695234e-02
 * time: 2.582103967666626
    13    -6.212233e+01     1.444631e-02
 * time: 2.671582937240

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671074
    AtomicLocal         -18.8557659
    AtomicNonlocal      14.8522619
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485374 
    Xc                  -19.3336822

    total               -62.261666461642
